In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

import edhec_risk_kit_204 as erk_204
import edhec_risk_kit_201 as erk_201

In [5]:
ind49_ret = erk_204.get_ind_returns(weighting="vw", n_inds=49)
ind49_ret = ind49_ret["1991":"2018"]

ind49_mcap = erk_204.get_ind_market_caps(49, weights=True)["1991":"2018"]

In [6]:
fff = erk_201.get_fff_returns()
fff.head()

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031


In [7]:
def get_CAPM_coef(ret, fff, start, end):
    excess_ret = ret.values - fff.loc[start:end, ['RF']].values.reshape(-1)
    mkt_excess = fff.loc[start:end,['Mkt-RF']]
    exp_var = mkt_excess.copy()
    exp_var["Constant"] = 1
    model = sm.OLS(excess_ret, exp_var).fit()
    return model.params[0]    

### 1.

In [10]:
beer_ret = ind49_ret['Beer']
get_CAPM_coef(beer_ret, fff, "1991", "2018")

0.5295417263363027

### 2.

In [11]:
steel_ret = ind49_ret['Steel']
get_CAPM_coef(steel_ret, fff, "1991", "2018")

1.554609599808529

### 3.

In [12]:
beer_ret_2 = ind49_ret['Beer']["2013":"2018"]
get_CAPM_coef(beer_ret_2, fff, "2013", "2018")

0.585960389056597

#### 4.

In [13]:
steel_ret_2 = ind49_ret['Steel']["2013":"2018"]
get_CAPM_coef(steel_ret_2, fff, "2013", "2018")

1.4169446523353808

#### 5. & 6.

In [14]:
cols = ind49_ret.columns
coefs = []
start = "1991"
end = "1993"
for c in cols:
    # 1991-1993 (both included) period
    c_ret = ind49_ret[c][start:end]
    coefs.append(get_CAPM_coef(c_ret, fff, start, end))

out = pd.DataFrame({'Index':cols, "Coef": coefs})
out.sort_values('Coef', ascending = True).head()

,Index,Coef
26,Gold,-0.553289
27,Mines,0.274762
30,Util,0.379527
28,Coal,0.529958
29,Oil,0.550604


In [19]:
out.sort_values('Coef', ascending = False).head()

,Index,Coef
10,Hlth,1.615195
35,Softw,1.608340
17,Cnstr,1.520882
9,Clths,1.516927
11,MedEq,1.473925


#### 7.

For the next 4 questions use the Full 3 Factor Fama-French model using the research data supplied in the following data file:
F-F_Research_Data_Factors.csv
and the same 1991-2018 period you have just used for the previous questions.    
Of the 49 industries, which displayed the highest Small Cap tilt when analyzed over the entire 1991-2018 period?

In [20]:
exp_var_2 = fff.loc["1991":"2018"]
exp_var_2.head()

,Mkt-RF,SMB,HML,RF
1991-01,0.0469,0.0379,-0.0184,0.0052
1991-02,0.0719,0.0395,-0.0054,0.0048
1991-03,0.0265,0.0389,-0.0123,0.0044
1991-04,-0.0028,0.0050,0.0142,0.0053
1991-05,0.0365,-0.0034,-0.0057,0.0047


In [21]:
def get_FF_coef(ret, fff, start, end):
    excess_ret = ret.values - fff.loc[start:end, ['RF']].values.reshape(-1)
    mkt_excess = fff.loc[start:end]
    exp_var = mkt_excess.copy()
    exp_var["Constant"] = 1
    model = sm.OLS(excess_ret, exp_var).fit()
#     print(model.params)
    return model.params[1],  model.params[2]

In [22]:
SMB = []
HML = []
for c in cols:
    # 1991-1993 (both included) period
    c_ret = ind49_ret[c]["1991":"2018"]
    SMB.append(get_FF_coef(c_ret, fff, "1991", "2018")[0])
    HML.append(get_FF_coef(c_ret, fff, "1991", "2018")[1])

out = pd.DataFrame({'Index':cols, "SMB": SMB, "HML":HML})
out.sort_values('SMB', ascending = False).head()

,Index,SMB,HML
19,FabPr,0.686027,0.496060
46,RlEst,0.680132,0.912121
28,Coal,0.544504,0.476809
15,Txtls,0.529438,0.997089
18,Steel,0.494700,0.315190


In [23]:
out.sort_values('SMB', ascending = True).head()

,Index,SMB,HML
3,Beer,-0.358381,0.005761
12,Drugs,-0.316796,-0.233712
4,Smoke,-0.309117,0.268762
1,Food,-0.268338,0.157499
45,Insur,-0.252436,0.572000


In [24]:
out.sort_values('HML', ascending = False).head()

,Index,SMB,HML
15,Txtls,0.529438,0.997089
46,RlEst,0.680132,0.912121
44,Banks,-0.186368,0.835839
22,Autos,0.203058,0.803970
24,Ships,0.162210,0.783453


In [25]:
out.sort_values('HML', ascending = True).head()

,Index,SMB,HML
35,Softw,0.071591,-0.863809
34,Hardw,0.201481,-0.698151
36,Chips,0.322006,-0.604432
37,LabEq,0.465954,-0.262712
12,Drugs,-0.316796,-0.233712
